# Monitor Task Decorator

The `monitor` decorator is designed for tasks that need to poll a specific state at regular intervals until a success criterion is met. This is useful for various scenarios, including time-based triggers, file existence checks, and monitoring other tasks or WorkGraphs.

## Built-in Monitors

- **TimeMonitor**: Start a task at a specified time.
- **FilMonitor**: Execute a task when a particular file exists.

## Behavior

While polling, the task sleeps for a specified interval (default 1.0 second, customizable by the user), allowing the WorkGraph engine to manage other tasks.

### Example Usage

#### Task Monitor

A task waits for another task in another WorkGraph to complete.


In [5]:

from aiida_workgraph import WorkGraph, task
from aiida import load_profile
load_profile()


@task.calcfunction()
def add(x, y, t=1):
    import time
    time.sleep(t.value)
    return x + y

wg1 = WorkGraph(name="test_task_monitor1")
# this task will wait for the "add2" task in the "test_task_monitor2" workgraph to finish
monitor1 = wg1.add_task("workgraph.task_monitor", workgraph_name="test_task_monitor2", task_name="add2")
add1 = wg1.add_task(add, "add1",x=1, y=2, t=0)
add1.waiting_on.add(monitor1)
wg1.submit()
#
wg2 = WorkGraph(name="test_task_monitor2")
# let add2 sleep for 5 seconds to simulate a calculation
wg2.add_task(add, "add2",x=1, y=2, t=5)
wg2.submit(wait=True)
wg1.wait()
print("ctime of add1: ", wg1.tasks.add1.node.ctime)
print("citme of add2: ", wg2.tasks.add2.node.ctime)
# calculate the time difference between the creation of the two task nodes
time_difference = wg1.tasks.add1.node.ctime - wg2.tasks.add2.node.ctime
assert time_difference.total_seconds() > 5

WorkGraph process created, PK: 108754
WorkGraph process created, PK: 108755
2024-08-16 16:28:45.741860+02:00
2024-08-16 16:28:40.044891+02:00



#### Time Monitor

A task waits until a specified time.



In [1]:
from aiida_workgraph import WorkGraph, task
import datetime


wg = WorkGraph(name="test_monitor")
# The task will wait until 2024-08-16, 10:54:00
monitor1 = wg.add_task("workgraph.time_monitor", time=datetime.datetime(2024, 8, 16, 10, 54, 0))
add1 = wg.add_task(add, x=1, y=2)
add1.waiting_on.add(monitor1)


#### File Monitor

Start a task when a specified file exists.


In [ ]:
wg = WorkGraph(name="test_monitor")
# The task will wait until the file exists, checking every 2 seconds.
monitor1 = wg.add_task("workgraph.file_monitor", filepath="/tmp/test.txt", interval=2.0)
add1 = wg.add_task(add, x=1, y=2)
add1.waiting_on.add(monitor1)


## Conclusion

These enhancements provide powerful tools for managing dependencies within WorkGraph.